In [ ]:
# Range of day_obs for data query
import rubin_nights.dayobs_utils as rn_dayobs

day_obs = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())
n_days = 30

# On-sky Efficiency {params.day_obs_min} to {params.day_obs_max}

In [ ]:
# Some configuration for password/RPS tokenfiles

import getpass
import os
import sys

# Who is running the notebook? Some of us have preferences ..
username = getpass.getuser()
# Where is the notebook running? (RSPs are 'special')
current_location = os.getenv("EXTERNAL_INSTANCE_URL", "")

# RUBIN_SIM_DATA_DIR at usdf
if "usdf" in current_location:
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# TOKEN CONFIGURATION
if current_location != "":
    # You are on an rsp.
    # You should use the default RSP values, whether summit/base/USDF.
    tokenfile = None
    site = None
# If you are outside of an RSP? - just use USDF and your own USDF-RSP token
# See https://rsp.lsst.io/guides/auth/creating-user-tokens.html
# See also rubin_nights.get_access_token for information about the default locations for tokens (or overrides)

In [ ]:
# Imports
import warnings
import copy
import logging

import numpy as np
import pandas as pd
import sqlite3
import healpy as hp
import matplotlib.pyplot as plt
import colorcet as cc
import skyproj
from IPython.display import display, HTML

from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord
import astropy.units as u

from rubin_scheduler.scheduler.utils import SchemaConverter
from rubin_scheduler.site_models import Almanac
from rubin_scheduler.scheduler.features import Conditions
from rubin_scheduler.utils import (
    ddf_locations,
    angular_separation,
    approx_ra_dec2_alt_az,
    Site,
    SURVEY_START_MJD,
)

import rubin_sim.maf as maf
from rubin_sim.data import get_baseline

from rubin_nights import connections
import rubin_nights.dayobs_utils as rn_dayobs
import rubin_nights.plot_utils as rn_plots
import rubin_nights.augment_visits as augment_visits
import rubin_nights.rubin_scheduler_addons as rn_sch
import rubin_nights.rubin_sim_addons as rn_sim
import rubin_nights.observatory_status as observatory_status
import rubin_nights.scriptqueue as scriptqueue
import rubin_nights.scriptqueue_formatting as scriptqueue_formatting
import rubin_nights.targets_and_visits as targets_and_visits

import importlib

from lsst_survey_sim import lsst_support, simulate_lsst, plot

band_colors = rn_plots.PlotStyles.band_colors
logging.getLogger("rubin_nights").setLevel(logging.INFO)

# %load_ext memory_profiler

In [ ]:
# Set up connections to data (will use consdb, exposure log and narrative log)
endpoints = connections.get_clients()

In [ ]:
day_obs_max = day_obs
day_obs_min = rn_dayobs.time_to_day_obs(
    rn_dayobs.day_obs_to_time(day_obs_max) - TimeDelta(n_days, format="jd")
)
day_obs_min = rn_dayobs.day_obs_str_to_int(day_obs_min)
print(f"Querying for lsstcam visits {day_obs_min} to {day_obs_max}")

one_day = TimeDelta(1, format="jd")
days = rn_dayobs.day_obs_to_time(day_obs_min) + one_day * np.arange(
    0,
    (rn_dayobs.day_obs_to_time(day_obs_max) - rn_dayobs.day_obs_to_time(day_obs_min)).jd
    + 1,
)
day_obs_list = [
    rn_dayobs.day_obs_str_to_int(rn_dayobs.time_to_day_obs(d)) for d in days
]

In [ ]:
# Get Consdb data and add some flags for bad visits, filter changes, and calculate slew times and identify expected overheads vs. excess

# We query all programs to get slewtimes and idle throughout entire night
# But these are what will be considered "science"
try:
    from rubin_nights.reference_values import SCIENCE_PROGRAMS

    programs = SCIENCE_PROGRAMS
except ImportError:
    programs = [
        "BLOCK-365",
        "BLOCK-407",
        "BLOCK-408",
        "BLOCK-416",
        "BLOCK-417",
        "BLOCK-419",
        "BLOCK-421",
    ]

# Just a flag to make it clear if we're skipping retrieval from the consdb.
# (this can make it much quicker to repeat this cell to tweak the inserts below)
refresh_visits = True
if refresh_visits:
    skip_imgtypes = ["bias", "flat", "dark"]
    query = (
        "select *, q.* from cdb_lsstcam.visit1 left join cdb_lsstcam.visit1_quicklook as q on visit1.visit_id = q.visit_id "
        f"where visit1.day_obs >= {day_obs_min} and visit1.day_obs <= {day_obs_max} and img_type != 'bias' and img_type != 'flat' and img_type != 'dark'"
    )
    visits = endpoints["consdb"].query(query)
    visits = augment_visits.augment_visits(visits, "lsstcam")
    visits.reset_index(inplace=True)
    visits.drop("index", axis=1, inplace=True)

dome_open = observatory_status.get_dome_open_close(
    rn_dayobs.day_obs_to_time(day_obs_min),
    rn_dayobs.day_obs_to_time(day_obs_max) + TimeDelta(1, format="jd"),
    endpoints["efd"],
)

wait_before_slew = 1.6
settle = 1.5
max_scatter = 10

if len(visits) > 0:

    cols = ["overhead", "fault_idle", "program_change", "filter_change", "bad_flag"]
    new_df = pd.DataFrame(
        np.zeros((len(visits), len(cols))), columns=cols, index=visits.index
    )
    visits = visits.merge(new_df, right_index=True, left_index=True)

    # Flag science_program changes
    program_change = np.where(
        (visits.science_program[:-1].values != visits.science_program[1:].values)
    )[0]
    program_change = program_change + 1
    pmask = np.zeros(len(visits))
    pmask[0] = 1
    pmask[program_change] = 1
    visits["program_change"] = pmask

    # Flag filter changes
    filter_change = np.where(
        (visits.band[:-1].values != visits.band[1:].values)
        & (visits.day_obs[:-1].values == visits.day_obs[1:].values)
    )[0]
    filter_change = filter_change + 1
    fmask = np.zeros(len(visits))
    fmask[filter_change] = 1
    visits["filter_change"] = fmask

    # calculate slew times and identify expected overheads
    visits, slewing = rn_sch.add_model_slew_times(
        visits,
        endpoints["efd"],
        model_settle=wait_before_slew + settle,
        dome_crawl=True,
        slew_while_changing_filter=False,
    )
    valid_overhead = np.min(
        [
            np.where(np.isnan(visits.slew_model.values), 0, visits.slew_model.values)
            + max_scatter,
            visits.visit_gap.values,
        ],
        axis=0,
    )
    visits["overhead"] = valid_overhead

    # Need to remove faults for the first visit of the night or where there was a different program we didn't fetch
    skipped_visits = np.concatenate(
        [
            np.array([0]),
            np.where(visits.visit_id[:-1].values + 1 != visits.visit_id[1:].values)[0]
            + 1,
        ]
    )

    fault = visits.visit_gap - valid_overhead
    fault[skipped_visits] = np.nan
    visits["fault_idle"] = fault

    visits.loc[skipped_visits, "model_gap"] = np.nan

    # Pull lsst-dm excluded visit list to flag bad visits
    bad_visit_ids = augment_visits.fetch_excluded_visits("lsstcam")
    visits["bad_flag"] = np.zeros(len(visits), int)
    idx = visits.query("visit_id in @bad_visit_ids").index
    visits.loc[idx, "bad_flag"] = 1
    # Also pull bad visit lists from exposure log
    ee = endpoints["exposure_log"].query_log(
        rn_dayobs.day_obs_to_time(day_obs_min), rn_dayobs.day_obs_to_time(day_obs_max)
    )
    if len(ee) > 0:

        def make_visit_id(x):
            return f"{x.day_obs:d}{x.seq_num:05d}"

        exp_log_bad_visit_ids = (
            ee.query("exposure_flag == 'junk'").apply(make_visit_id, axis=1).values
        )
        if len(exp_log_bad_visit_ids) > 0:
            idx = visits.query("visit_id in @exp_log_bad_visit_ids").index
            visits.loc[idx, "bad_flag"] = 1

    # And flag close_loop visits as 'bad' for purposes of efficiency
    idx = visits.query("observation_reason.str.contains('close_loop')").index
    visits.loc[idx, "bad_flag"] = 1

    sci = visits.query("science_program in @programs")
    print(
        "all visits:",
        len(visits),
        "science visits:",
        len(visits.query("science_program in @programs")),
    )

else:
    print("Found no visits")
    print("The remainder of this notebook requires visits.")

## -- need to mark time associated with bad visits as fault somehow ..

In [ ]:
# Setting this up early makes it easier to slot in all other values
night_times = []
for day_obs in day_obs_list:
    dome_day = dome_open.query("day_obs == @day_obs")
    if (len(dome_day) == 0) or ("sunset12" not in dome_day.columns):
        sunset, sunrise = rn_dayobs.day_obs_sunset_sunrise(day_obs, -12)
        night_hours = (sunrise.mjd - sunset.mjd) * 24
        open_hours = 0
    else:
        sunset = Time(dome_day.sunset12.iloc[0]).tai
        sunrise = Time(dome_day.sunrise12.iloc[0]).tai
        night_hours = dome_day.night_hours.iloc[0]
        open_hours = dome_day.open_hours.sum()
    closed_hours = night_hours - open_hours
    night_times.append(
        pd.Series(
            {
                "sunset": sunset.tai.mjd,
                "sunrise": sunrise.tai.mjd,
                "night_hours": night_hours,
                "open_hours": open_hours,
                "closed_hours": closed_hours,
            },
            name=day_obs,
        )
    )
night_times = pd.DataFrame(night_times)

In [ ]:
# Plot expected slewtime vs. actual visit gap
# Sometimes we want a subset of days ..

print("Look at the visit gaps and expected visit gaps, for SCIENCE visits only:")

dayobs = visits.day_obs.unique()

q = visits.query("science_program in @programs and day_obs in @dayobs")
total_time = (q.shut_time.sum() + q.overhead.sum() + q.fault_idle.sum()) / 3600
total_onsky = q.exp_time.sum() / 3600
total_req = (q.shut_time.sum() + q.overhead.sum()) / 3600
total_fault_idle = q.fault_idle.sum() / 3600
dd = pd.DataFrame(
    [total_time, total_onsky, total_req, total_fault_idle, len(q), len(q) * 30 / 3600],
    index=[
        "time for visits",
        "onsky exptime",
        "onsky+overhead",
        "fault+idle_sci",
        "nvis",
        "estimate time onsky",
    ],
    columns=["all " + "_".join(programs)],
)
display(dd.T)

print(
    f"Min/median/mean predicted overheads: {q.overhead.min():.2f} {q.overhead.median():.2f} {q.overhead.mean():.2f}"
)
print(
    f"Min/median/mean actual visit gaps: {q.visit_gap.min():.2f} {q.visit_gap.median():.2f} {q.visit_gap.mean():.2f}"
)

fig, axes = plt.subplots(1, 3, figsize=(20, 5))
ax = axes[0]
ax.plot(q.visit_gap, q.slew_model, "k.")
# ax.hexbin(q.visit_gap, q.slew_model, gridsize=200, extent=[0, 30, 0, 30], mincnt=1, vmin=0, vmax=10)
x = np.arange(0, 500, 1)
ax.plot(x, x, alpha=0.3)
ax.fill_betweenx(x1=x + max_scatter, x2=x, y=x, color="pink", alpha=0.2)
ax.set_xlim(0, 30)
ax.set_ylim(0, 30)
ax.grid(alpha=0.4)
ax.set_xlabel("Visit gap (seconds)")
ax.set_ylabel("Predicted visit gap (seconds)")

ax = axes[1]
ax.plot(q.visit_gap, q.slew_model, "k.")
x = np.arange(0, 500, 1)
ax.plot(x, x, alpha=0.3)
ax.fill_betweenx(x1=x + max_scatter, x2=x, y=x, color="pink", alpha=0.2)
# ax.set_xlim(0, 30)
# ax.set_ylim(0, 30)
ax.grid(alpha=0.4)
ax.set_xlabel("Visit gap (seconds)")
ax.set_ylabel("Predicted visit gap (seconds)")

ax = axes[2]
ax.plot(q.slew_distance, q.model_gap, "k.")
ax.set_ylim(-10, 10)
ax.grid(alpha=0.3)
ax.set_xlabel("Slew distance (deg)")
_ = ax.set_ylabel("visit_gap - prediction (seconds)")

In [ ]:
print("Compare expected visit gap (without faults) to the v5.1 model visit gap:")
q = visits.query("science_program in @programs")


def slew_ratios(g):
    slew_eff = (g.exp_time.sum()) / (g.dark_time.sum() + g.overhead.sum())
    ideal_eff = (g.exp_time.sum()) / (g.dark_time.sum() + g.slew_model_ideal.sum())
    med_gap = np.nanmedian((g.obs_start_mjd - g.prev_obs_end_mjd) * 24 * 60 * 60)
    time_lost = (g.overhead.sum() - g.slew_model_ideal.sum()) / 60
    return pd.Series(
        {
            "slew_eff": slew_eff,
            "ideal_eff": ideal_eff,
            "slew_ratio": slew_eff / ideal_eff,
            "med_gap": med_gap,
            "slew_time_loss": time_lost,
        }
    )


visit_gap_eff = q.groupby("day_obs")[
    [
        "exp_time",
        "dark_time",
        "overhead",
        "slew_model_ideal",
        "obs_start_mjd",
        "prev_obs_end_mjd",
    ]
].apply(slew_ratios)

night_times = night_times.join(visit_gap_eff, how="outer")

display(visit_gap_eff.round(2))

print("mean values")
print(f"Current max open-shutter-efficiency: {visit_gap_eff.slew_eff.mean(): 0.2f}")
print(
    f"Ideal model open-shutter-efficiency equivalent: {visit_gap_eff.ideal_eff.mean(): 0.2f}"
)
print(f"Ratio - slew / ideal {visit_gap_eff.slew_ratio.mean() :0.2f}")

In [ ]:
# Get narrative time lost logs - rely on OSs to only report values for night time
time_lost_logs = endpoints["narrative_log"].query_log(
    rn_dayobs.day_obs_to_time(day_obs_min),
    rn_dayobs.day_obs_to_time(day_obs_max),
    {"min_time_lost": "0.00001"},
)
if len(time_lost_logs) > 0:
    time_lost_logs = time_lost_logs.query("not component.str.contains('AuxTel')")

    def time_to_day_obs(x):
        return rn_dayobs.day_obs_str_to_int(
            rn_dayobs.time_to_day_obs(Time(x.date_begin, format="isot", scale="utc"))
        )

    time_lost_logs["day_obs"] = time_lost_logs.apply(time_to_day_obs, axis=1)
    log_fault = (
        time_lost_logs.query("time_lost_type == 'fault'")
        .groupby("day_obs")
        .agg({"time_lost": "sum"})
        .rename({"time_lost": "log_fault"}, axis=1)
    )
    log_weather = (
        time_lost_logs.query("time_lost_type == 'weather'")
        .groupby("day_obs")
        .agg({"time_lost": "sum"})
        .rename({"time_lost": "log_weather"}, axis=1)
    )
    log_lost = log_fault.merge(log_weather, how="outer", on="day_obs")
else:
    log_lost = None

if log_lost is not None:
    night_times = night_times.join(log_lost, how="outer")

In [ ]:
# Calculate values relating to visits and expected visit gaps

fault_gap = 5 * 60  # seconds


def time_active(g):
    nn = night_times.loc[g.name]

    # Estimate of all fault/idle time - expect higher idle during other surveys
    all_fault_idle = round(g.fault_idle.sum() / 60 / 60, 2)
    # Estimate all fault/idle time - but only where visit_gap > 5 minutes
    gap_fault_idle = round(
        g.query("visit_gap > @fault_gap").fault_idle.sum() / 60 / 60, 2
    )

    # Estimate of time 'missing' from science at start or end of the night
    fbs = g.query("science_program in @programs")
    if len(fbs) == 0:
        twi_to_start = (nn.sunrise - nn.sunset) * 24
        end_to_twi = 0
    else:
        twi_to_start = (fbs.obs_start_mjd.min() - nn.sunset) * 24
        end_to_twi = (nn.sunrise - fbs.obs_end_mjd.max()) * 24

    # Estimate time spent in FBS compared to time expected for these visits in FBS
    time_in_fbs = (fbs.obs_end_mjd.max() - fbs.obs_start_mjd.min()) * 24
    fbs_no_close_loop = fbs.query(
        "science_program in @programs and not observation_reason.str.contains('close_loop')"
    )
    time_predict_in_fbs = (
        (fbs_no_close_loop.dark_time.sum() + fbs_no_close_loop.slew_model_ideal.sum())
        / 60
        / 60
    )
    fbs_open_shutter_fraction = (fbs.exp_time.sum()) / (nn.night_hours * 60 * 60)

    return pd.Series(
        {
            "nvis": len(g),
            "nvis_sci": len(fbs),
            "delay_start": twi_to_start,
            "early_end": end_to_twi,
            "total_fault_idle": all_fault_idle,
            "total_fault_idle_gap": gap_fault_idle,
            "time_in_fbs": time_in_fbs,
            "time_predict_in_fbs": time_predict_in_fbs,
            "fbs_open_shutter": fbs_open_shutter_fraction,
        }
    )


fault_eff = visits.groupby("day_obs").apply(time_active, include_groups=False)

night_times = night_times.join(fault_eff, how="outer")

In [ ]:
# Make some extrapolations for efficiency over the whole night

# Estimate the fraction of the night used/run
ratio_active = round(
    (
        night_times.night_hours
        - night_times.delay_start
        - night_times.early_end
        - night_times.total_fault_idle_gap
    )
    / (night_times.night_hours),
    2,
)

# Overall efficiency if we had used the whole night, minus faults, with slew efficiency at that night
eff_all = round(ratio_active * night_times.slew_ratio, 2)


# For FBS extrapolation, the FBS open shutter fraction already includes faults within the FBS and we don't want to include other faults
ratio_avail = (
    night_times.night_hours - night_times.delay_start - night_times.early_end
) / night_times.night_hours

eff_fbs = round(
    night_times.time_predict_in_fbs / night_times.time_in_fbs * ratio_avail, 2
)


# An estimate of the simulation effectiveness ratio - using unscheduled downtime only
eff_sim_ref = round((night_times.night_hours - 0.37) / night_times.night_hours, 2)

effs = pd.DataFrame(
    [ratio_active, ratio_avail, eff_all, eff_fbs, eff_sim_ref],
    index=["ratio_active", "ratio_avail", "eff_all", "eff_fbs", "eff_sim_ref"],
    columns=day_obs_list,
).T

night_times = night_times.join(effs, how="outer")

In [ ]:
print(
    f"fault+idle (hours) - total: {night_times.total_fault_idle.sum():.2f} mean: {np.nanmean(night_times.total_fault_idle):.2f}"
)
print(
    f"fault+idle gap (hours) - total: {night_times.total_fault_idle_gap.sum():.2f} mean: {np.nanmean(night_times.total_fault_idle_gap):.2f}"
)
print(
    f"log fault (hours) - total: {night_times.log_fault.sum():.2f} mean: {np.nanmean(night_times.log_fault):.2f}"
)

# make an average ratio, with a fudge factor for time lost at ends of night
print(f"System availability estimate all: {np.nanmean(night_times.eff_all):.2f}")
print(f"System availability estimate fbs: {np.nanmean(night_times.eff_fbs):.2f}")

In [ ]:
print("Using baseline_v5.1.0_10yrs as the comparison")
fraction_wfd = 1692518.00 / 2075536.00
fO_comparison = 762.00 / 825 * 0.9 / fraction_wfd
print(f"fraction WFD in this sim: {fraction_wfd:.2f}")
print(f"median_nvis / 825 * 0.9 / fraction_wfd = {fO_comparison:.2f}")

In [ ]:
# Plot of above system availabity * fO_comparison info per night
plt.figure(figsize=(12, 6))
q = night_times[["eff_fbs", "eff_all"]].fillna(0)
x = np.arange(0, len(q))
y = q["eff_fbs"] * fO_comparison
plt.plot(x, y, marker=".", label="efficiency fbs visits")
y = q["eff_all"] * fO_comparison
plt.plot(x, y, marker=".", label="efficiency all visits")
yy = np.nanmean(
    np.concatenate(
        [
            q["eff_all"].values * fO_comparison,
            q["eff_fbs"].values * fO_comparison,
        ]
    )
)
plt.axhline(yy, color="gray", linestyle=":", label="mean")
_ = plt.xticks(x, q.index, rotation=90)
_ = plt.ylabel("SA*fO")
_ = plt.legend(loc=(1.01, 0.5))
plt.grid(alpha=0.2)

plt.figure(figsize=(8, 6))
y = q["eff_fbs"]
_ = plt.hist(y, bins=30, alpha=0.5, label="efficiency fbs visits")
y = q["eff_all"]
_ = plt.hist(y, bins=30, alpha=0.5, label="efficiency all visits")
plt.xlabel("SA * fO")
plt.ylabel("Nnights")
_ = plt.legend()